# Huffman Code

- **Created by Andrés Segura Tinoco**  
- **Created on June 20, 2019**

## Metrics
- Time
- Size
- Distribution type: normal, uniform
- Entropy
- Parallelism

<hr>
<p><a href="https://github.com/ansegura7/Icaro/">« Home</a></p>